In [1]:
import bs4
from IPython.display import IFrame, HTML
import os
import pandas as pd
import posixpath
import re
import requests
# from requests.compat import urlparse
from requests.compat import urljoin, urlparse
# %maptlotlib inline

# CS 109A/STAT 121A/AC 209A/CSCI E-109A: Final

**Harvard University**<br/>
**Fall 2017**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine

----

Web scraping component of Crime project.

Goal: Scrape data from UCI crime tables and census.

In [2]:
baseurl = 'https://ucr.fbi.gov/ucr-publications'
req = requests.get(baseurl)
soup = bs4.BeautifulSoup(req.text, 'html.parser')

In [3]:
# Get links matching a 4-digit year and check if they are for the "Crime in the US" Report
year_links = soup.find_all(text=re.compile('\d{4}'), name='a')
crime_main_urls = []
for link in year_links:
    if 'crime-in-the-u.s' in link['href'] or 'cius' in link['href']:
        crime_main_urls.append(link['href'])
crime_main_urls

['https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016',
 'https://ucr.fbi.gov/crime-in-the-u.s/2015/crime-in-the-u.s.-2015',
 'https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/cius-home',
 'https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/cius-home',
 'https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012',
 'https://ucr.fbi.gov/crime-in-the-u.s/2011/crime-in-the-u.s.-2011',
 'https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010',
 'http://www2.fbi.gov/ucr/cius2009/rankingmessage.htm',
 'http://www2.fbi.gov/ucr/cius2008/index.html',
 'http://www2.fbi.gov/ucr/cius2007/index.html',
 'http://www2.fbi.gov/ucr/cius2006/index.html',
 'http://www2.fbi.gov/ucr/05cius/',
 'http://www2.fbi.gov/ucr/cius_04/',
 'https://ucr.fbi.gov/crime-in-the-u.s/2003',
 'https://ucr.fbi.gov/crime-in-the-u.s/2002',
 'https://ucr.fbi.gov/crime-in-the-u.s/2001',
 'https://ucr.fbi.gov/crime-in-the-u.s/2000',
 'https://ucr.fbi.gov/crime-in-the-u.s/1

In [4]:
def navigate_to_msa_table(year_url):
    '''Follow a prescribed garden path to find the MSA table url for a given year report'''
    #strip the current link to just the home page
    if year_url.endswith('rankingmessage.htm'):
        year_url = year_url.replace('/rankingmessage.htm', '')
    if year_url.endswith('index.html'):
        year_url = year_url.replace('/index.html', '')
    #fetch and parse the page for this year
    req = requests.get(year_url)
    soup = bs4.BeautifulSoup(req.text, 'html.parser')
    #pull out the (first) link that mentions Violent Crime and clean the link
    crime_url = soup.find(string=re.compile('Violent'), name='a')['href']
    if not crime_url.startswith('http'):
        crime_url = '/'.join([year_url, crime_url])
    if crime_url.endswith('index.html'):
        crime_url = crime_url.replace('/index.html', '/')
    #fetch and parse the Violent Crime page
    req = requests.get(crime_url)
    soup = bs4.BeautifulSoup(req.text, 'html.parser')
    #pull out the link for metropolitain statistical areas
    msa_url = soup.find(string=re.compile('Metropoliti?an Statistical Area'), name='a')['href']
    print("found:", msa_url)
#     if not msa_url.endswith('/'):
#         msa_url += '/'
    if msa_url.startswith('.'):
        msa_url = crime_url + msa_url

    return resolveComponents(msa_url)


def resolveComponents(url):
    """
    Mostly copied from https://stackoverflow.com/a/4317446/218118
    
    >>> resolveComponents('http://www.example.com/foo/bar/../../baz/bux/')
    'http://www.example.com/baz/bux/'
    >>> resolveComponents('http://www.example.com/some/path/../file.ext')
    'http://www.example.com/some/file.ext'
    """

    parsed = urlparse(url)
    new_path = posixpath.normpath(parsed.path)
    if parsed.path.endswith('/'):
        # Compensate for issue1707768
        new_path += '/'
    cleaned = parsed._replace(path=new_path)
    return cleaned.geturl()

msa_urls = []

for year_url in crime_main_urls:
#     print(year_url)
    try:
        msa_urls.append(navigate_to_msa_table(year_url))
    except TypeError:
#         import pdb; pdb.set_trace()
        print('-'*10 + ' Error with top-level url - {}'.format(year_url))
msa_urls

found: https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-4
found: https://ucr.fbi.gov/crime-in-the-u.s/2015/crime-in-the-u.s.-2015/tables/table-6
found: https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-6
found: https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/6tabledatadecpdf/table-6
found: https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/6tabledatadecpdf
found: https://ucr.fbi.gov/crime-in-the-u.s/2011/crime-in-the-u.s.-2011/tables/table-6
found: https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/table-6
found: ../../data/table_06.html
found: ../../data/table_06.html
found: ../../data/table_06.html
found: ../../data/table_06.html
---------- Error with top-level url - http://www2.fbi.gov/ucr/05cius/
---------- Error with top-level url - http://www2.fbi.gov/ucr/cius_04/
---------- Error with top-level url - https://ucr.fbi.gov/crime-in-the-u.s/2003
---------- Error

['https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-4',
 'https://ucr.fbi.gov/crime-in-the-u.s/2015/crime-in-the-u.s.-2015/tables/table-6',
 'https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-6',
 'https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/6tabledatadecpdf/table-6',
 'https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/6tabledatadecpdf',
 'https://ucr.fbi.gov/crime-in-the-u.s/2011/crime-in-the-u.s.-2011/tables/table-6',
 'https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/table-6',
 'http://www2.fbi.gov/ucr/cius2009/data/table_06.html',
 'http://www2.fbi.gov/ucr/cius2008/data/table_06.html',
 'http://www2.fbi.gov/ucr/cius2007/data/table_06.html',
 'http://www2.fbi.gov/ucr/cius2006/data/table_06.html']

In [5]:
# https://www2.fbi.gov/ucr/cius2009/offenses/violent_crime/index.html
# https://www2.fbi.gov/ucr/cius2009/offenses/violent_crime/index.html

In [6]:
# %pdb

The format of the table:

* "Major" rows are 1 per MSA. This takes many differnt forms in the html. 

Per MSA:

* Row 1: the header (MSA name) and total population
* Row 2: Empty except counties' names in column 2 (but this changes depending on format of html headers)
* Row 3-X: full stats for major cities (one row per city)
* 4 (generally): full stats for total reporting
* 5 (when not 100% fully reporting): full stats for estimated 100% reporting
* 5: Normalized rate per 100,000 of total reporting


In [7]:
year_pat = re.compile('(\d{4})')
def parse_msa_table(table_url):
    # Retrieve the table
    req = requests.get(table_url)
    soup = bs4.BeautifulSoup(req.text, 'html.parser')
    tab1 = soup.find(name='table', class_='data')
    
    # Extract column names
    header_cells = tab1.find('thead').find_all('th')[1:]
    table_cols = [re.sub('\s+', ' ', t.get_text(separator=' ').strip()) for t in header_cells]

    # Parse the MSA info and create a dataframe
    rows = tab1.find('tbody').find_all('tr')
    msa_list = []
    current_msa = None
    for row in rows:
        try:
            current_msa = parse_msa_row(row, current_msa, msa_list, table_cols) 
        except TypeError as e:
            # Some rows are empty, containing only empty tds with no th cells. Continue to the next row in that case.
            print(e)
            continue


    # Combine each MSA's stats into a single large dataframe
    all_msa_df = pd.concat(msa_list)

    # Read the year from the url and add it to the df
    year = year_pat.search(table_url).groups()[0]
    all_msa_df['year'] = year
    all_msa_df = all_msa_df.convert_objects(convert_numeric=True)
    
    # Collapse and rename some problem columns (inconsistent across years)
    cols = all_msa_df.columns
    larceny_col = cols[cols.to_series().str.contains('Larceny', case=False)][0]
    all_msa_df.rename(columns={larceny_col: 'Larceny'}, inplace=True)

    rape_col = cols[cols.to_series().str.contains('Rape', case=False)][0]
    all_msa_df.rename(columns={rape_col: 'Rape'}, inplace=True)

    return all_msa_df


def parse_msa_row(row, current_msa, msa_list, table_cols):
    '''Parse a row and adjust the current msa as appropriate.
    
    Since the MSA object is really a set of rows, and those rows aren't well defined (MDs) interspersed,
    the row parser needs the current row, the current msa (to adjust attributes), and the list of 
    all MSAs (MSA dicts are only added to the list when the previous MSA has been completed and a new
    header has been found). Updates MSA list and current MSA in place; returns None '''
    row_type = determine_row_type(row)
    
    if row_type == 'header':
        if current_msa:
            msa_df = msa_to_df(current_msa, table_cols)
            msa_list.append(msa_df)
        current_msa = parse_header_row(row)
    elif row_type == 'county_list':
        counties = parse_county_row(row)
        current_msa['counties'] = counties
    elif row_type == 'table_row':
        counts = parse_numerical_row(row)
        current_msa['counts'].append(counts)
    else:
        raise StandardError('Unrecognized row type: {}'.format(row_type))
    return current_msa


def determine_row_type(row):
    '''Scan a html row and determine which function should parse it.
    
    Primary row types:
    - 
    - 
    '''
    th = row.find('th')
    if len(th['class']):
        if in_list(th['class'], ['subguide1', 'subguide2', 'group0']) and len(th.text.strip()):
            # subguide1 is for MSA; subguide2 is for MD (Metropolitan Division), smaller breakdowns of MSAs
            # In later years, "groupN" is the 0-indexed column number. Group 0 is the first column, or header
            # In 2010, a th in col0 appears for some numeric columns.
            row_type = 'header'
        # Check if any item in the header's classes is a member of the following header groups
        # This list comprehension is the fastest way to check intersection
        else:  # in_list(th['class'], ['subguide1e', 'group1']):
            if 'Includes' in th.text or 'Includes' in row.find('td').text:
                row_type = 'county_list'
            else:
                row_type = 'table_row'
    return row_type


def in_list(query_list, target_list):
    '''Return True if any member of ListA is in ListB'''
    return bool(len(list([i for i in query_list if i in target_list])))

def parse_header_row(row):
    th = row.find('th')
    msa_label = th.text.strip()
    try:
        pop_idx = 0
        msa_pop = parse_pop(row, pop_idx)
    except ValueError:
        pop_idx = 1
        msa_pop = parse_pop(row, pop_idx)

    new_msa = dict(msa_label=msa_label, msa_pop=msa_pop, counts=[])
    return new_msa


def parse_pop(row, pop_idx):
    return int(row.find_all('td')[pop_idx].text.strip().replace(',', ''))


def parse_county_row(row):
    counties = row.find('th').text.strip()
    return counties


def parse_numerical_row(row):
    label = row.find('th').text.strip()
    cols = [column_value.text.replace(',', '').replace('%', '').strip() for column_value in row.find_all('td')]
    if len(label):
        cols.insert(0, label)
    return cols


def msa_to_df(current_msa, table_cols):
    df = pd.DataFrame([c for c in current_msa['counts']], columns=table_cols)
    for k in set(current_msa.keys()) - set(['counts']):
        df[k] = current_msa[k]
    return df.convert_objects(convert_numeric=True)

msa_dfs = []
for url in msa_urls:
    print(url)
    msa_dfs.append(parse_msa_table(url))

all_year_dfs = pd.concat(msa_dfs, ignore_index=True)
all_year_dfs.describe()

https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-4


/Users/kastman/miniconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:132: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
/Users/kastman/miniconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:31: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


https://ucr.fbi.gov/crime-in-the-u.s/2015/crime-in-the-u.s.-2015/tables/table-6
https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-6
https://ucr.fbi.gov/crime-in-the-u.s/2013/crime-in-the-u.s.-2013/tables/6tabledatadecpdf/table-6
https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/6tabledatadecpdf
https://ucr.fbi.gov/crime-in-the-u.s/2011/crime-in-the-u.s.-2011/tables/table-6
https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/table-6
http://www2.fbi.gov/ucr/cius2009/data/table_06.html
http://www2.fbi.gov/ucr/cius2008/data/table_06.html
'NoneType' object is not subscriptable
http://www2.fbi.gov/ucr/cius2007/data/table_06.html
http://www2.fbi.gov/ucr/cius2006/data/table_06.html


,Aggravated assault,Burglary,Larceny,Motor vehicle theft,Murder and nonnegligent manslaughter,Population,Property crime,Rape,Robbery,Violent crime,msa_pop,year
count,16089.000000,16143.000000,16134.000000,15829.000000,16250.000000,1.250600e+04,15984.000000,16023.000000,16250.000000,15877.000000,1.626200e+04,16262.000000
mean,1199.271229,2722.242588,8553.961869,1279.867528,25.359766,8.201184e+04,12351.146059,130.973082,663.704874,1908.408673,1.305731e+06,2011.046796
std,3161.785847,6223.325592,20774.493302,3865.121598,74.907758,3.140353e+05,30061.588206,292.966997,2297.674180,5628.398022,2.620349e+06,3.143743
min,0.000000,4.000000,6.000000,0.000000,0.000000,5.320000e+01,94.000000,0.000000,0.000000,0.000000,3.315600e+04,2006.000000
25%,147.800000,451.000000,1602.350000,114.000000,2.000000,1.000000e+02,2213.000000,23.000000,49.225000,227.000000,1.541390e+05,2008.000000
50%,303.800000,842.000000,2491.000000,235.700000,5.000000,1.506650e+04,3554.400000,42.200000,109.850000,444.000000,3.370110e+05,2011.000000
75%,790.000000,2041.100000,5825.500000,630.000000,15.000000,7.441975e+04,8229.500000,101.000000,313.975000,1122.000000,1.006870e+06,2014.000000
max,44100.000000,68227.000000,264374.000000,75245.000000,1092.000000,8.566917e+06,357850.000000,4968.000000,37129.000000,82359.000000,2.017799e+07,2016.000000


In [8]:
all_year_dfs.head()

,Aggravated assault,Burglary,Counties/principal cities,Larceny,Motor vehicle theft,Murder and nonnegligent manslaughter,Population,Property crime,Rape,Robbery,Violent crime,counties,msa_label,msa_pop,year
0,335.0,905.0,City of Abilene,3113.0,251.0,8.0,122523.0,4269.0,70.0,133.0,546.0,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016
1,387.0,1104.0,Total area actually reporting,3530.0,298.0,11.0,100.0,4932.0,79.0,137.0,614.0,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016
2,227.8,649.9,"Rate per 100,000 inhabitants",2077.9,175.4,6.5,NaN,2903.1,46.5,80.6,361.4,"Includes Callahan, Jones, and Taylor Counties","Abilene, TX M.S.A.",169885,2016
3,555.0,2483.0,City of Akron,5483.0,680.0,34.0,197257.0,8646.0,212.0,415.0,1216.0,Includes Portage and Summit Counties,"Akron, OH M.S.A.2",703561,2016
4,833.0,3865.0,Total area actually reporting,12599.0,989.0,41.0,96.2,17453.0,335.0,530.0,1739.0,Includes Portage and Summit Counties,"Akron, OH M.S.A.2",703561,2016


The primary output of the scraping is a **AC209_RawScraped.csv**, a single csv table where all year's data is stored. This is more or less a mirror of all the tables in all the reports, regardless of row type, Metro/Micro statistical area, etc.

In [9]:
all_year_dfs.to_csv('data/derived/AC209_RawScraped.csv', encoding='utf_8')